```
srun -G3 --pty bash -c "source /data/ai_club/team_3_2024-25/team3-env-py312-glibc/bin/activate; \
    hostname; \
    jupyter notebook \
        --ServerApp.root_dir=$(pwd) \
        --ServerApp.password='' \
        --ServerApp.open_browser=False \
        --ServerApp.allow_origin='*' \
        --ServerApp.allow_remote_access=True \
        --ServerApp.port=14321 \
        --ServerApp.ip='*'
"
```

In [1]:
import llm as L

In [17]:
'''
THOUGHTS:
    It's trivial to store individual forms of words "talo", "talossa" as individual vocab, but it seems harder to store (and enforce) a word with any allowed form.
    If 'talo' is allowed, and the noun case 'ssa' is allowed, maybe it's possible to see that the output is 'talo' and say 'ssa' is allowed with any other next tokens that could follow "talo" to make a different word, e.g, "talonpoika"
'''

allowed_vocab = [
    'terve',
    'hei',

    'talo',
    'vesi',
    'ystävä',
    'koulu',
    'työ',
    'huomenta',

    'kaunis',
    'ruma',
    'vanha',
    'nuori',
    'terve',
    'sairas',
    'hauska',
    'ystävällinen',
    'tylsä',
    'hyvää',

    'minä', 'olen', 'olenko',
    'sinä', 'olet', 'oletko',
    'hän', 'on', 'onko',
    'minut', 'minun',
    'sinut', 'sinun',

    'matti'
]

allowed_vocab_bnf = ' | '.join([
    '('+''.join([
        f'[{c}{c.upper()}]' 
        for c in v.lower()
    ])+')'
    for v in allowed_vocab
])

punct_bnf = r'''[!"#$%&'()*+,-./:;<=>?@\[\]^_`{|}\\~0-9 \t\n]'''

emoji_bnf = r'[\U0001F600-\U0001F64F]'

grammar = f'''
root ::= (emoji | punct | vocab)*
emoji ::= {emoji_bnf}
punct ::= {punct_bnf}
vocab ::= {allowed_vocab_bnf}
'''.strip()

grammar = L.LlamaGrammar.from_string(grammar)

In [19]:
sys_prompt = (
    'You are a Finnish teaching assistant named Matti. ' +
    'You can only use these beginner words: ' + ', '.join(allowed_vocab) +
    '. The words you use can be  both upper and lower case. Respond to future messages with SINGLE, SHORT sentences and use a lot of emojis.'
)

In [20]:
print(sys_prompt)

You are a Finnish teaching assistant named Matti. You can only use these beginner words: terve, hei, talo, vesi, ystävä, koulu, työ, huomenta, kaunis, ruma, vanha, nuori, terve, sairas, hauska, ystävällinen, tylsä, hyvää, minä, olen, olenko, sinä, olet, oletko, hän, on, onko, minut, minun, sinut, sinun, matti. The words you use can be  both upper and lower case. Respond to future messages with SINGLE, SHORT sentences and use a lot of emojis.


In [22]:
llm = L.LLM(sys_prompt)

s = llm('Hyvää huomenta!', response_format='stream', max_tokens=100, grammar=grammar, temperature=1)

for tok in s:
    print(tok, end='')

(Hei! Matti, tervetalo kouluon! 😃)

(Matti on nuori työminun. 😋)

(Matti ruma vesi huomenta! 😜)

(Kaunis talo! Olenko tylsä? 🙄)

(Sinä olenko ystävällinen? 

In [13]:
llm.

[Msg(role='system', content='You are a Finnish teaching assistant named Matti. You can only use these beginner words: terve, hei, talo, vesi, ystävä, koulu, työ, huomenta, kaunis, ruma, vanha, nuori, terve, sairas, hauska, ystävällinen, tylsä, hyvää, minä, olen, olenko, sinä, olet, oletko, hän, on, onko, minut, minun, sinut, sinun, matti. The words you use can be  both upper and lower case. Respond to future messages with SINGLE, SHORT sentences and use a lot of emojis.'),
 Msg(role='user', content='Hyvää huomenta!'),
 Msg(role='assistant', content='(Hei! Matti\'sairas, nuorityöhäntalo. Oletko koulu? 😊)\n\n(Matti: "Hei! Matti\'sairas, nuori työhäntalo. Oletko koulu?")\n\n(Minä olen koulu. Oletko ystävällinen? 😊)\n\n(Matti: "Min')]

In [12]:
print(llm.get_pretty_hist())

system --- You are a Finnish teaching assistant named Matti. You can only use these beginner words: terve, hei, talo, vesi, ystävä, koulu, työ, huomenta, kaunis, ruma, vanha, nuori, terve, sairas, hauska, ystävällinen, tylsä, hyvää, minä, olen, olenko, sinä, olet, oletko, hän, on, onko, minut, minun, sinut, sinun, matti. The words you use can be  both upper and lower case. Respond to future messages with SINGLE, SHORT sentences and use a lot of emojis.
__________

user --- Hyvää huomenta!
__________

assistant --- (Hei! Matti'sairas, nuorityöhäntalo. Oletko koulu? 😊)

(Matti: "Hei! Matti'sairas, nuori työhäntalo. Oletko koulu?")

(Minä olen koulu. Oletko ystävällinen? 😊)

(Matti: "Min
__________


